In [304]:
import pandas as pd 
import numpy as np

# DT 문제 1번

#### ※함수에 들어가 있는 변수나 flow는 본인이 바꾸셔도 가능하며 결과만 똑같이 나오면 됩니다!
#### ※hard코딩(이 데이터셋에만 적용되는 코딩방법) 말고 전체 데이터에 적용 가능하게 함수를 짜주셔야 합니다.

# Data Loading

In [305]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


# 1. Gini 계수에 대한 함수 만들기.

- Input은 Dataframe과 label이름으로 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다!

In [22]:
len(pd_data)

14

In [27]:
def get_gini(df, label):
    gini = 1
    for i in df[label].unique():
        gini -= (len(df.loc[df[label] == i])/len(df)) ** 2
    return gini

In [28]:
get_gini(pd_data, 'class_buys_computer')

0.4591836734693877

# 2. Feature의 Class를 이진 분류로 만들기
## ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})
- Powerset은 모든 조합을 뱉어내게 됩니다. 이건 그냥 완성된걸 드릴게요.

In [32]:
from itertools import chain, combinations

def powerset(feature_class):
    listed_data = list(feature_class)
    chain_set = chain.from_iterable(combinations(listed_data, i) for i in range(len(listed_data)+1))
    return [set_data for set_data in chain_set]

In [33]:
powerset(pd_data.age.unique())

[(),
 ('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior'),
 ('youth', 'middle_aged', 'senior')]

- 저 위에 부분 집합 중 우리가 원하는 집합(이진 분류)만 골라 내야하겠죠?
- 그 함수를 get_binary_split로 완성해주세요!
- 완성된 내용은 다음과 같이 나와야 합니다.

In [51]:
for i in powerset(pd_data['age'].unique()):
    print(len(i))

0
1
1
1
2
2
2
3


In [158]:
def get_binary_split(df, attribute):
    result = [i for i in powerset(df[attribute].unique()) if (len(i)!=0) & (len(i)!=len(df[attribute].unique()))]
    return result

In [159]:
get_binary_split(pd_data, "age")

[('youth',),
 ('middle_aged',),
 ('senior',),
 ('youth', 'middle_aged'),
 ('youth', 'senior'),
 ('middle_aged', 'senior')]

# 3. 다음은 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 함수를 사용하여 만들어주세요!
- DataFrame의 index를 사용하여 만들면 굉장히 편합니다..! 예시를 아래에서 보여드릴게요.
- 결과는 아래 아래 아래 줄과 같아야 합니다.

In [73]:
A = set([1,2,3])
B = set([3,4,5])
A.union(B) #A와 B의 합집합
pd_data.loc[A.union(B)]

,age,income,student,credit_rating,class_buys_computer
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no


In [161]:
def get_attribute_gini_index(df, attribute, label):
    result = {}
    for i in get_binary_split(df, attribute):
        d1 = get_gini(df.loc[df[attribute].isin(i)], label)
        d2 = get_gini(df.loc[~df[attribute].isin(i)], label)
        gini = len(df.loc[df[attribute].isin(i)])/len(df)*d1 + len(df.loc[~df[attribute].isin(i)])/len(df)*d2
        result["".join(list(i))] = gini
    return result

In [162]:
get_attribute_gini_index(pd_data, "age", "class_buys_computer")

{'youth': 0.3936507936507937,
 'middle_aged': 0.35714285714285715,
 'senior': 0.45714285714285713,
 'youthmiddle_aged': 0.45714285714285713,
 'youthsenior': 0.35714285714285715,
 'middle_agedsenior': 0.3936507936507937}

- 여기서 가장 작은값으로 분류를 해야겠죠?

In [199]:
min(get_attribute_gini_index(pd_data, "age", "class_buys_computer").items())

('middle_aged', 0.35714285714285715)

# 다음의 문제를 위에서 작성한 함수를 통해 구한 값으로 보여주세요!
## 문제1) income의 이진분류를 얻는 함수 get_binary_split(pd_data, "income")을 통해 보여주세요.

In [164]:
get_binary_split(pd_data, "income")

[('high',),
 ('medium',),
 ('low',),
 ('high', 'medium'),
 ('high', 'low'),
 ('medium', 'low')]

## 문제2) 가장 Gini계수가 높은 Feature 즉 분류를 하는데 가장 중요한 변수를 선정하시고 get_attribute_gini_index함수를 통해 Gini index를 제시해주세요.

In [256]:
def get_min_feature(pd_data):
    df = pd_data.columns.tolist()
    del df[df.index('class_buys_computer')]
    max_gini = 1
    min_feature = {}
    for i in df:
        min_feature[i] = min(get_attribute_gini_index(pd_data, i, "class_buys_computer").items(),default=0)
    return min_feature.items()
get_min_feature(pd_data)
#가장 gini index가 높은 feature : age
# split_point : middle_aged
# gini_index : 0.35714285714285715

dict_items([('age', ('middle_aged', 0.35714285714285715)), ('income', ('high', 0.4428571428571429)), ('student', ('no', 0.3673469387755103)), ('credit_rating', ('excellent', 0.42857142857142855))])

## 문제3) 2에서 구한 Feature로 DataFrame을 분류 해주시고 나눠진 2개의 클래스에서 각각 다음으로 중요한 Feature를 선정해주시고 Gini index를 제시해주세요.

In [257]:
print(get_min_feature(pd_data.loc[pd_data['age']=='middle_aged']))
#모든 label이 동일
print(get_min_feature(pd_data.loc[pd_data['age']!='middle_aged']))
#가장 gini index가 높은 feature : student
# split_point : no
# gini_index : 0.31999999999999984

dict_items([('age', 0), ('income', ('high', 0.0)), ('student', ('no', 0.0)), ('credit_rating', ('excellent', 0.0))])
dict_items([('age', ('senior', 0.48)), ('income', ('high', 0.375)), ('student', ('no', 0.31999999999999984)), ('credit_rating', ('excellent', 0.4166666666666667))])
